In [1]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [3]:
load_dotenv()

True

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

C:\Users\madan\AppData\Local\Temp\ipykernel_3112\1867371969.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)


In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001EC778DE780>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001EC778FC440>, temperature=0.5, model_kwargs={}, openai_api_key='sk-krNzG6Dl8C7OEgivkX5FBDcuptT3qLsToY1pH74fOtT3BlbkFJb2EQ75uupXQ_DxuZbVxNwgQdGSVcIu1pkqHFspo20A', openai_proxy='')

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\madan\AppData\Local\Temp\ipykernel_3112\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [15]:
from pathlib import Path
file_path=r"C:\Users\madan\mcqgen\data.txt"

In [16]:
file_path

'C:\\Users\\madan\\mcqgen\\data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine lea

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [21]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [22]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoreti

C:\Users\madan\AppData\Local\Temp\ipykernel_3112\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert english grammarian and writer. Given a Multiple Choice Quiz for data science students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:

{
    "1": {
        "mcq": "What is machine learning?",
        "options": {
            "a": "A field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data",
            "b": "A field of study in natural language processing",
            "c": "A field of study in computer vision",
            "d": "A field of study in medicine"
        },
        "correct": "a"
    },
    

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2064
Prompt Tokens:1607
Completion Tokens:457
Total Cost:0.0033245


In [24]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for descri

In [25]:
quiz_str=response.get("quiz")

In [26]:
quiz_dict = json.loads(quiz_str)

In [27]:
quiz_dict

{'1': {'mcq': 'What is machine learning?',
  'options': {'a': 'A field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data',
   'b': 'A field of study in natural language processing',
   'c': 'A field of study in computer vision',
   'd': 'A field of study in medicine'},
  'correct': 'a'},
 '2': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Donald Hebb',
   'b': 'Alan Turing',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '3': {'mcq': "What is the synonym used for 'machine learning' in the early days?",
  'options': {'a': 'Self-teaching computers',
   'b': 'Deep learning',
   'c': 'Data mining',
   'd': 'Predictive analytics'},
  'correct': 'a'},
 '4': {'mcq': 'What is the foundation of machine learning?',
  'options': {'a': 'Artificial intelligence',
   'b': 'Statistics and mathematical optimization',
   'c': 'Data mining',
   'd': 'Predictive analytics'},
  

In [28]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [29]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data | b: A field of study in natural language processing | c: A field of study in computer vision | d: A field of study in medicine',
  'Correct': 'a'},
 {'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Alan Turing | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': "What is the synonym used for 'machine learning' in the early days?",
  'Choices': 'a: Self-teaching computers | b: Deep learning | c: Data mining | d: Predictive analytics',
  'Correct': 'a'},
 {'MCQ': 'What is the foundation of machine learning?',
  'Choices': 'a: Artificial intelligence | b: Statistics and mathematical optimization | c: Data mining | d: Predictive analytics',
  'Correct': 'b'},
 {'MCQ': 'What is the purpose of modern-day machine learning?',
  'Choices': 'a: To classif

In [30]:
df=pd.DataFrame(quiz_table_data)

In [31]:
df

,MCQ,Choices,Correct
0,What is machine learning?,a: A field of study in artificial intelligence...,a
1,Who coined the term 'machine learning'?,a: Donald Hebb | b: Alan Turing | c: Arthur Sa...,c
2,What is the synonym used for 'machine learning...,a: Self-teaching computers | b: Deep learning ...,a
3,What is the foundation of machine learning?,a: Artificial intelligence | b: Statistics and...,b
4,What is the purpose of modern-day machine lear...,a: To classify data based on models which have...,c


In [32]:
df.to_csv("ML_Science_Quiz.csv",index=False)